# 1. GEE - CAMS PM 2.5 history
Uses the CAMS dataset on Google Earth Engine
and the location of our cities of interest
to compute the mean pollution level at each city,
for different points in time. Also calculates
the historical average for the available period.

In [110]:
import ee
import geemap
import geetools
import pandas as pd

In [111]:
# Function to add 'date' property (just the day, ignoring hours)
def add_date(image):
    
    date = ee.Date(image.get('model_initialization_datetime')).format('YYYY-MM-dd');
    year = ee.Date(image.get('model_initialization_datetime')).get('year')
    month = ee.Date(image.get('model_initialization_datetime')).get('month')
    week = ee.Date(image.get('model_initialization_datetime')).get('week')
    day = ee.Date(image.get('model_initialization_datetime')).get('day')
    
    forecast_hour = image.get('model_forecast_hour');
    
    
    return image.set({
        "date": date,
        "year": year,
        "month": month,
        "week": week,
        "day": day
    })

In [112]:
def monthly_historical_mean(month):
    
    filtered = cams_collection.filter(ee.Filter.eq('month', month))
    mean = filtered.mean()
    
    return mean.set('month', month)

In [113]:
def weekly_historical_mean(week):
    
    filtered = cams_collection.filter(ee.Filter.eq('week', week))
    mean = filtered.mean()
    
    return mean.set('week', week)

In [114]:
def recent_daily_means(date):
    
    filtered = cams_filtered.filter(ee.Filter.eq('date', date))
    mean = filtered.mean()
    
    return mean.set('date', date)

In [115]:
ee.Initialize()

In [116]:
# All available CAMS images, within a 24h forecast and a 0h UTC
cams_collection = (ee.ImageCollection('ECMWF/CAMS/NRT')
                   .filterDate("2017-01-01", "2023-12-31")
                   .filter(ee.Filter.eq('model_initialization_hour', 0))
                   .filter(ee.Filter.lte('model_forecast_hour', 21))
                   .select('particulate_matter_d_less_than_25_um_surface'))

In [117]:
# The images from the time which we will analyze
start_date = '2024-08-10'
end_date = '2024-09-12'

cams_filtered = (ee.ImageCollection('ECMWF/CAMS/NRT')
                   .filterDate(start_date, end_date)
                   .filter(ee.Filter.eq('model_initialization_hour', 0))
                   .filter(ee.Filter.lte('model_forecast_hour', 21))
                   .select('particulate_matter_d_less_than_25_um_surface'))

In [118]:
# Apply the add_date function to each image in the collection
cams_collection = cams_collection.map(add_date)
cams_filtered = cams_filtered.map(add_date)

In [119]:
# First, let's compute the mean for every month and week, at every pixel
unique_months = cams_collection.aggregate_array('month').distinct()
unique_weeks = cams_collection.aggregate_array('week').distinct()

monthly_history = ee.ImageCollection(unique_months.map(monthly_historical_mean))
weekly_history = ee.ImageCollection(unique_weeks.map(weekly_historical_mean))

In [120]:
monthly_size = monthly_history.size().getInfo()
monthly_history = monthly_history.toList(monthly_size)

In [129]:
# Exports each entry in the image
def export_monthly_image(image, month):
    
    # Get the image ID (or generate one if the ID is empty)
    # Create export task for the image
    task = ee.batch.Export.image.toAsset(
        image=image,
        description=f'Month_{month}_Mean',
        assetId=f'projects/ee-tree-coverage-cities/assets/pm2p5-monthly-2017-2023/Month_{month}_Mean',  # Replace with your asset path
        region=[-90, -60, 30, 15], 
        scale=44528
    )

    # Start the export task
    task.start()
    print(f'Exporting image for month {month}')

In [130]:
for i, month in enumerate(range(1,13)):
    image = ee.Image(monthly_history.get(i))
    export_monthly_image(image, month)

Exporting image for month 1
Exporting image for month 2
Exporting image for month 3
Exporting image for month 4
Exporting image for month 5
Exporting image for month 6
Exporting image for month 7
Exporting image for month 8
Exporting image for month 9
Exporting image for month 10
Exporting image for month 11
Exporting image for month 12


In [134]:
# Access the september data
september_history = ee.Image("projects/ee-tree-coverage-cities/assets/pm2p5-monthly-2017-2023/Month_9_Mean")

In [135]:
# Compute the daily means for the interval of interest
unique_dates = cams_filtered.aggregate_array('date').distinct()
recent_daily_means = ee.ImageCollection(unique_dates.map(recent_daily_means))

In [136]:
# Load the cities feature collection and filter for South American countries
southAmericaISO = ['ARG', 'BOL', 'BRA', 'CHL', 'COL', 'ECU', 'GUY', 'PRY', 'PER', 'SUR', 'URY', 'VEN'];
cities = ee.FeatureCollection("projects/dw-city-tree-coverage/assets/city_outlines")\
              .filter(ee.Filter.inList('CTR_MN_ISO', southAmericaISO))\
              .filter(ee.Filter.gte('P15', 2e5));


In [137]:
# Reduces of the recent images over the cities, getting an average of the territory
def daily_mean_for_cities(image):
    
    # Use reduceRegions to calculate the mean PM2.5 over each city
    
    means = image.reduceRegions(
        collection=cities,                  # The cities feature collection
        reducer=ee.Reducer.mean(),          # Use mean reducer
        scale=44528,                        # Adjust scale based on the dataset resolution
    )
    
    # Add the 'date' property from the image to each feature (city)
    means_with_date = means.map(lambda feature: feature.set('date', image.get('date')))
    return means_with_date

# Apply the mean_for_cities function to all images in the dailyMeans ImageCollection
city_recent_daily_means = recent_daily_means.map(daily_mean_for_cities).flatten()

In [146]:
# Apply the mean_for_cities function to all images in the dailyMeans ImageCollection
city_september_history = september_history.reduceRegions(
        collection=cities,                  # The cities feature collection
        reducer=ee.Reducer.mean(),          # Use mean reducer
        scale=44528,                        # Adjust scale based on the dataset resolution
    ).map(lambda feature: feature.set('month', september_history.get('month')))

In [147]:
# Export the tree to Google Drive
feature_collections = [
    city_september_history,
    city_recent_daily_means,
]

# Define corresponding filenames for the exports
filenames = [
    'city_september_history',
    'city_recent_daily_means',
]

# Loop through each FeatureCollection and export it as a CSV file to Google Drive
for fc, filename in zip(feature_collections, filenames):
    task = ee.batch.Export.table.toDrive(
        collection=fc,
        description=filename,
        fileFormat='CSV',  # Export format
        folder='GEE_Exports_PollutionSouthAmerica',  # Google Drive folder (change if necessary)
    )
    
    # Start the export task
    task.start()

    # Print confirmation
    print(f"Exporting {filename} as CSV to Google Drive.")


Exporting city_september_history as CSV to Google Drive.
Exporting city_recent_daily_means as CSV to Google Drive.


In [148]:
september_history.getInfo()

{'type': 'Image',
 'bands': [{'id': 'particulate_matter_d_less_than_25_um_surface',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [300, 193],
   'crs': 'EPSG:4326',
   'crs_transform': [0.4000018297127405,
    0,
    -90.00041168536661,
    0,
    -0.4000018297127405,
    15.20006952908414]}],
 'version': 1726089536529169,
 'id': 'projects/ee-tree-coverage-cities/assets/pm2p5-monthly-2017-2023/Month_9_Mean',
 'properties': {'month': 9,
  'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-30.000137136213453, -62.20165264017084],
    [-15.050068927421446, -62.201652655178236],
    [7.375033801253107, -62.20165263119034],
    [30.654128989693984, -62.17599187719717],
    [30.213473887662133, 15.393535273841623],
    [22.325102103106865, 15.40083034359519],
    [-0.10000045242140376, 15.400830395540064],
    [-15.050068808848824, 15.400830349886375],
    [-30.000137218956564, 15.400830386362106],
    [-52.42523977274817, 15.400830398443647],
   